In [2]:
dfInflacaoAnual = spark.sql("SELECT * FROM POC_Lakehouse.BC_Inflacao_Anual")
display(dfInflacaoAnual)

StatementMeta(, 1609418b-cc4d-4e13-b66f-c606049e40cc, 4, Finished, Available)

SynapseWidget(Synapse.DataFrame, cf0fca03-b0bc-4c75-add1-d28c2c246e32)

In [3]:
dfInflacaoPredicoes = spark.sql("SELECT * FROM POC_Lakehouse.BC_Inflacao_Predicoes")
dfInflacaoPredicoes = dfInflacaoPredicoes.withColumnRenamed("MediaAnualPredita","MediaAnual")
# display(dfInflacaoPredicoes)

StatementMeta(, 1609418b-cc4d-4e13-b66f-c606049e40cc, 5, Finished, Available)

In [4]:
dfInflacao = dfInflacaoAnual.union(dfInflacaoPredicoes)
# display(dfInflacao)

StatementMeta(, 1609418b-cc4d-4e13-b66f-c606049e40cc, 6, Finished, Available)

In [5]:
# nome_tabela = "BC_InflacaoUnionSpark"
# dfInflacao.write.mode("overwrite").format("delta").save("Tables/" + nome_tabela)

StatementMeta(, 1609418b-cc4d-4e13-b66f-c606049e40cc, 7, Finished, Available)

In [6]:
dfIndicadorPorItem = spark.sql("SELECT * FROM POC_Lakehouse.IndicadorPorItem")
# display(dfIndicadorPorItem)

StatementMeta(, 1609418b-cc4d-4e13-b66f-c606049e40cc, 8, Finished, Available)

In [7]:
from pyspark.sql.functions import year, when, lit, col
from datetime import datetime

dfItens = spark.sql("SELECT * FROM POC_Lakehouse.Itens")
dfItens = dfItens.withColumn("Ano", year(dfItens["DataInicio"]))
dfItens = dfItens.withColumn("AnoInicioProjecao", dfItens["Ano"]+1)

current_year = datetime.now().year
dfItens = dfItens.withColumn("Ano", when(col("Ano").isNull(), lit(current_year)).otherwise(col("Ano")))
dfItens = dfItens.withColumn("AnoInicioProjecao", when(col("AnoInicioProjecao").isNull(), lit(current_year+1)).otherwise(col("AnoInicioProjecao")))

# display(dfItens)

StatementMeta(, 1609418b-cc4d-4e13-b66f-c606049e40cc, 9, Finished, Available)

In [8]:
dfAnos = spark.range(2022, 2034).select(col("id").alias("anoProjecao"))
# display(dfAnos)

StatementMeta(, 1609418b-cc4d-4e13-b66f-c606049e40cc, 10, Finished, Available)

In [13]:
# Join entre dfItens e dfIndicadorPorItem
dfIndicadorPorItem = dfIndicadorPorItem.withColumnRenamed("Codigo","CodigoIndicadorPorItem")
dfJoinItensIndicadores = dfItens.join(dfIndicadorPorItem, dfItens.Codigo == dfIndicadorPorItem.CodigoItem, 'left_outer')
dfJoinAnosProjecao =  dfJoinItensIndicadores.crossJoin(dfAnos)

# # Join entre df_merged e dfInflacao
dfInflacao = dfInflacao.withColumnRenamed("Ano","AnoInflacao")
dfInflacao = dfInflacao.withColumnRenamed("Indicador","IndicadorInflacao")
dfJoinInflacao = dfJoinAnosProjecao.join(dfInflacao, (dfJoinAnosProjecao.Indicador == dfInflacao.IndicadorInflacao) & (dfJoinAnosProjecao.anoProjecao == dfInflacao.AnoInflacao), 'left_outer')
# display(dfJoinInflacao)

dfBaseProjecao = dfJoinInflacao.drop("Codigo", "Tipo", "Setor", "CodigoIndicadorPorItem", "anoProjecao", "IndicadorInflacao", "ValorEntrada").orderBy("CodigoItem", "AnoInflacao")
display(dfBaseProjecao)

StatementMeta(, 1609418b-cc4d-4e13-b66f-c606049e40cc, 15, Finished, Available)

SynapseWidget(Synapse.DataFrame, b310b527-c6a3-486a-8dba-c50aaffd5a2b)

In [15]:
from pyspark.sql.window import Window
from pyspark.sql.functions import sum as pyspark_sum

# Defina a janela particionada por CodigoItem e ordene por AnoInflacao
windowSpec = Window.partitionBy("CodigoItem").orderBy("AnoInflacao").rowsBetween(Window.unboundedPreceding, 0)

# Calcule o valor projetado para cada ano e, em seguida, o acumulado usando a janela definida
dfBaseProjecao = dfBaseProjecao.withColumn("ProjecaoReajuste", col("ValorTotal") * (col("MediaAnual") / 100))
dfBaseProjecao = dfBaseProjecao.withColumn("ProjecaoReajusteAcumulado", pyspark_sum("ProjecaoReajuste").over(windowSpec))
dfBaseProjecao = dfBaseProjecao.withColumn("ValorProjetadoReajustado", col("ProjecaoReajusteAcumulado") + col("ValorTotal"))


display(dfBaseProjecao)


StatementMeta(, 1609418b-cc4d-4e13-b66f-c606049e40cc, 17, Finished, Available)

SynapseWidget(Synapse.DataFrame, 30bccd93-2e99-4197-b371-561bac5718d7)

In [17]:
nome_tabela = "BC_InflacaoProjetadda"
dfBaseProjecao.write.mode("overwrite").format("delta").save("Tables/" + nome_tabela)

StatementMeta(, 1609418b-cc4d-4e13-b66f-c606049e40cc, 19, Finished, Available)